In [1]:
from dotenv import load_dotenv

assert load_dotenv("../../.env")

In [2]:
import ragna

ragna.__version__

'0.1.dev29+g03721c7.d20231009084220'

In [3]:
from ragna import demo_config, Config

demo_config

Config(local_cache_root=PosixPath('/home/philip/.cache/ragna'), state_database_url='sqlite://', queue_database_url='memory', ragna_api_url='http://127.0.0.1:31476', ragna_ui_url='http://127.0.0.1:31477', document_class=<class 'ragna.core.LocalDocument'>, upload_token_secret='9709d3f424d472672ccb97882812d33c66ac30df3669cfbd299cc80996c96a17', upload_token_ttl=30, registered_source_storage_classes={'Ragna/DemoSourceStorage': <class 'ragna.source_storage._demo.RagnaDemoSourceStorage'>}, registered_assistant_classes={'Ragna/DemoAssistant': <class 'ragna.assistant._demo.RagnaDemoAssistant'>})

In [4]:
document_path = "demo_document.txt"

with open(document_path, "w") as file:
    file.write("Ragna is an open-source RAG orchestration app\n")

In [5]:
from ragna.core import Rag
from ragna.assistant import (
    RagnaDemoAssistant,
    OpenaiGpt35Turbo16kAssistant,
    OpenaiGpt4Assistant,
)
from ragna.source_storage import (
    ChromaSourceStorage,
    RagnaDemoSourceStorage,
    LanceDBSourceStorage,
)

rag = Rag(demo_config)

async with rag.chat(
    documents=[document_path],
    source_storage=RagnaDemoSourceStorage,
    assistant=RagnaDemoAssistant,
) as chat:
    prompt = "What is Ragna?"
    answer = await chat.answer(prompt)

print(answer)

I can't really help you with your prompt:

> What is Ragna?

I can at least show you the sources that I was given:

- demo_document.txt: Ragna is an open-source RAG orchestration app


In [6]:
import itertools
import asyncio
from pprint import pprint

source_storages = [ChromaSourceStorage, LanceDBSourceStorage]
assistants = [OpenaiGpt35Turbo16kAssistant, OpenaiGpt4Assistant]


async def answer_prompt(source_storage, assistant):
    async with rag.chat(
        documents=[document_path],
        source_storage=source_storage,
        assistant=assistant,
    ) as chat:
        return await chat.answer(prompt)


experiments = {
    (source_storage.display_name(), assistant.display_name()): answer_prompt(
        source_storage, assistant
    )
    for source_storage, assistant in itertools.product(source_storages, assistants)
}

pprint(experiments)

{('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): <coroutine object answer_prompt at 0x7f61babd4b40>,
 ('Chroma', 'OpenAI/gpt-4'): <coroutine object answer_prompt at 0x7f61babd4bc0>,
 ('LanceDB', 'OpenAI/gpt-3.5-turbo-16k'): <coroutine object answer_prompt at 0x7f61babd4c40>,
 ('LanceDB', 'OpenAI/gpt-4'): <coroutine object answer_prompt at 0x7f61babd4cc0>}


In [7]:
results = dict(zip(experiments.keys(), await asyncio.gather(*experiments.values())))
pprint(results)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Unhandled exception in task 1907504c-778f-479c-a660-5eb1d9084404.
Traceback (most recent call last):
  File "/home/philip/.conda/envs/ragna-dev/lib/python3.9/site-packages/huey/api.py", line 382, in _execute
    task_value = task.execute()
  File "/home/philip/git/ora/ragna/core/_queue.py", line 34, in execute
    return execute(*self.args)
  File "/home/philip/git/ora/ragna/core/_queue.py", line 29, in execute
    return fn(self, *args, **kwargs)
  File "/home/philip/git/ora/ragna/source_storage/_lancedb.py", line 94, in retrieve
    return list(
  File "/home/philip/git/ora/ragna/utils/_misc.py", line 39, in take_sources_up_to_max_tokens
    for source in sources:
  File "/home/philip/git/ora/ragna/source_storage/_lancedb.py", line 97, in <genexpr>
    Source(
  File "pydantic/main.py", line 341, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for Source
location
  none is not an allowed value (type=type_error.none.not_allowed)
Unhandle

RagnaException: Task failed, error=ValidationError(model='Source', errors=[{'loc': ('location',), 'msg': 'none is not an allowed value', 'type': 'type_error.none.not_allowed'}]), retries=0, traceback=NoneType: None
, task_id=1907504c-778f-479c-a660-5eb1d9084404, component=LanceDB, action=retrieve